# Similarity Search

## Level 3

So far we experimented with different visualization techniques on the results, t-SNE and PCA on the results. Now we will calculate the accuracies of the features obtained from the pretrained and finetuned models.

In [13]:
import numpy as np
import pickle
from tqdm import notebook
import random
import time
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import PIL
from PIL import Image
from sklearn.neighbors import NearestNeighbors
import glob
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

For these experiments we will use the same features of the Caltech101 dataset that we were using before.

Let's utilize the features from the previously trained model.

In [2]:
filenames = pickle.load(open('data/filenames-caltech101.pickle', 'rb'))
feature_list = pickle.load(open('data/features-caltech101-resnet.pickle', 'rb'))
class_ids = pickle.load(open('data/class_ids-caltech101.pickle', 'rb'))

num_images = len(filenames)
num_features_per_image = len(feature_list[0])
print("Number of images = ", num_images)
print("Number of features per image = ", num_features_per_image)

Number of images =  8677
Number of features per image =  2048


First, let's make a helper function that calculates the accuracy of the resultant features using the nearest neighbors brute force algorithm.

In [14]:
def classname(str):
    return str.split('/')[-2]


def classname_filename(str):
    return str.split('/')[-2] + '/' + str.split('/')[-1]


def calculate_accuracy(feature_list):
    num_nearest_neighbors = 5
    correct_predictions = 0
    incorrect_predictions = 0
    neighbors = NearestNeighbors(n_neighbors=num_nearest_neighbors,
                                 algorithm='brute',
                                 metric='euclidean').fit(feature_list)
    for i in notebook.tqdm(range(len(feature_list))):
        distances, indices = neighbors.kneighbors([feature_list[i]])
        for j in range(1, num_nearest_neighbors):
            if (classname(filenames[i]) == classname(
                    filenames[indices[0][j]])):
                correct_predictions += 1
            else:
                incorrect_predictions += 1
    print(
        "Accuracy is ",
        round(
            100.0 * correct_predictions /
            (1.0 * correct_predictions + incorrect_predictions), 2))

## 1. Accuracy of Brute Force over Caltech101 features

In [15]:
calculate_accuracy(feature_list[:])

  0%|          | 0/8677 [00:00<?, ?it/s]

Accuracy is  88.46


## 2. Accuracy of Brute Force over the PCA compressed Caltect101 features

In [16]:
num_feature_dimensions = 100
pca = PCA(n_components=num_feature_dimensions)
pca.fit(feature_list)
feature_list_compressed = pca.transform(feature_list[:])

Let's calculate accuracy over the compressed features.

In [17]:
calculate_accuracy(feature_list_compressed[:])

  0%|          | 0/8677 [00:00<?, ?it/s]

Accuracy is  88.58


## 3. Accuracy of Brute Force over the finetuned Caltech101 features

In [18]:
filenames = pickle.load(open('data/filenames-caltech101.pickle', 'rb'))
feature_list = pickle.load(open('data/features-caltech101-resnet-finetuned.pickle', 'rb'))
class_ids = pickle.load(open('data/class_ids-caltech101.pickle', 'rb'))

In [20]:
num_images = len(filenames)
num_features_per_image = len(feature_list[0])
print("Number of images = ", num_images)
print("Number of features per image = ", num_features_per_image)

Number of images =  8677
Number of features per image =  101


In [21]:
calculate_accuracy(feature_list[:])

  0%|          | 0/8677 [00:00<?, ?it/s]

Accuracy is  89.35


## 4. Accuracy of Brute Force over the PCA compressed finetuned Caltech101 features

In [22]:
num_feature_dimensions = 100
pca = PCA(n_components=num_feature_dimensions)
pca.fit(feature_list)
feature_list_compressed = pca.transform(feature_list[:])

In [ ]:
calculate_accuracy(feature_list_compressed[:])

  0%|          | 0/8677 [00:00<?, ?it/s]